<a href="https://colab.research.google.com/github/Thieffenat/FreshFruit/blob/master/BenGraham2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing

In [0]:
!pip3 install iexfinance

     |████████████████████████████████| 51kB 2.7MB/s 


In [0]:
#test functionality
from iexfinance.stocks import Stock

IEX_TOKEN='pk_69cd67cc6f374f389b03ecb9cc0ade3a'

aapl = Stock('AAPL',token=IEX_TOKEN)
aapl.get_key_stats()

{'avg10Volume': 74189366.9,
 'avg30Volume': 49065952.47,
 'beta': 1.263710621201779,
 'companyName': 'Apple, Inc.',
 'day200MovingAvg': 246.28,
 'day30ChangePercent': -0.23355,
 'day50MovingAvg': 306.12,
 'day5ChangePercent': -0.141162,
 'dividendYield': 0.012246706683317891,
 'employees': 137000,
 'exDividendDate': '2020-02-07',
 'float': 4368956159,
 'marketcap': 1086125400400,
 'maxChangePercent': 244.7723,
 'month1ChangePercent': -0.23591,
 'month3ChangePercent': -0.097838,
 'month6ChangePercent': 0.134766,
 'nextDividendDate': None,
 'nextEarningsDate': '2020-05-05',
 'peRatio': 20.13,
 'sharesOutstanding': 4375480000,
 'ttmDividendRate': 3.04,
 'ttmEPS': 12.7549,
 'week52change': 0.36607799999999996,
 'week52high': 327.85,
 'week52low': 170.27,
 'year1ChangePercent': 0.366078,
 'year2ChangePercent': 0.3793,
 'year5ChangePercent': 1.0085,
 'ytdChangePercent': -0.173531}

In [0]:
#get stocks data
#define variables
#integrate alpaca

# Install Packages

In [0]:
!pip3 install pandas

In [0]:
#retrieve data test get_symbols
from iexfinance.refdata import get_symbols

get_symbols(output_format='pandas', token="sk_084ecc26ad39496b8a4acfeea1e5015a")

,symbol,exchange,name,date,type,iexId,region,currency,isEnabled,figi,cik
0,A,NYS,Agilent Technologies Inc.,2020-03-13,cs,IEX_46574843354B2D52,US,USD,True,BBG000C2V3D6,1090872
1,AA,NYS,Alcoa Corp.,2020-03-13,cs,IEX_4238333734532D52,US,USD,True,BBG00B3T3HD3,1675149
2,AAAU,PSE,Perth Mint Physical Gold ETF,2020-03-13,et,IEX_474B433136332D52,US,USD,True,BBG00LPXX872,None
3,AACG,NAS,ATA Creativity Global Sponsored ADR,2020-03-13,ad,IEX_44595A4C53392D52,US,USD,True,BBG000V2S3P6,None
4,AADR,PSE,AdvisorShares Dorsey Wright ADR ETF,2020-03-13,et,IEX_5253355435362D52,US,USD,True,BBG000BDYRW6,None
...,...,...,...,...,...,...,...,...,...,...,...
8848,ZUO,NYS,Zuora Inc. Class A,2020-03-13,cs,IEX_4D474631525A2D52,US,USD,True,BBG000BT3HG5,1423774
8849,ZVO,NAS,Zovio Inc.,2020-03-13,cs,IEX_514A435352362D52,US,USD,True,BBG000C3CQP1,1305323
8850,ZYME,NYS,Zymeworks Inc.,2020-03-13,cs,IEX_5253315838322D52,US,USD,True,BBG00GD15LJ5,1403752
8851,ZYNE,NAS,Zynerba Pharmaceuticals Inc,2020-03-13,cs,IEX_4E3154424A382D52,US,USD,True,BBG007BBS8B7,1621443


In [0]:
#retrieve data test for aapl stock
  from iexfinance.data_apis import get_data_points

get_data_points('AAPL',output_format='pandas', token="sk_084ecc26ad39496b8a4acfeea1e5015a").head

<bound method NDFrame.head of                       NEXTDIVIDENDDATE  ...                        ZIP
key                   NEXTDIVIDENDDATE  ...                        ZIP
weight                               1  ...                          1
description                             ...                        zip
lastUpdated  2020-02-07T16:24:55+00:00  ...  2020-03-13T12:58:20+00:00

[4 rows x 130 columns]>

In [0]:
!pip3 install pylivetrader

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Main

In [0]:
from pylivetrader import *

def initialize(context):
    # Define sectors we're interested in trading.
    context.sectors = [
        'Basic Materials',
        'Consumer Cyclical',
        'Financial Services',
        'Real Estate',
        'Consumer Defensive',
        'Healthcare',
        'Utilities',
        'Communication Services',
        'Energy',
        'Industrials',
        'Technology'
    ]

    context.months_until_rebalance = 1
    schedule_function(try_rebalance,
                      date_rule=date_rules.month_start(),
                      time_rule=time_rules.market_open())

In [0]:
#Rebalancing schedule
def try_rebalance(context, data):
    # See if it's time to rebalance every month.
    # We'll reevaluate our positions once every three months.
    if context.months_until_rebalance == 1:
        update_target_securities(context)
        rebalance(context)
        context.months_until_rebalance = 3
    else:
        context.months_until_rebalance -= 1

In [0]:
import pandas as pd

def build_sector_fundamentals(sector):
    '''
    In this method, for the given sector, we'll get the data we need for each stock
    in the sector from IEX. Once we have the data, we'll check that the earnings
    reports meet our criteria with `eps_good()`. We'll put stocks that meet those
    requirements into a dataframe along with all the data about them we'll need.
    '''
    stocks = get_sector(sector)
    if len(stocks) == 0:
        raise ValueError("Invalid sector name: {}".format(sector))

    # If we can't see its PE here, we're probably not interested in a stock. Omit it from batch queries.
    stocks = [s for s in stocks if s['peRatio'] is not None]

    # IEX doesn't like batch queries for more than 100 symbols at a time.
    # We need to build our fundamentals info iteratively.
    batch_idx = 0
    batch_size = 99
    fundamentals_dict = {}
    while batch_idx < len(stocks):
        symbol_batch = [s['symbol']
                        for s in stocks[batch_idx:batch_idx+batch_size]]
        stock_batch = Stock(symbol_batch)

        # Pull all the data we'll need from IEX.
        financials_json = stock_batch.get_financials()
        quote_json = stock_batch.get_quote()
        stats_json = stock_batch.get_key_stats()
        earnings_json = stock_batch.get_earnings()

        for symbol in symbol_batch:
            # We'll filter based on earnings first to keep our fundamentals info a bit cleaner.
            if not eps_good(earnings_json[symbol]):
                continue

            # Make sure we have all the data we'll need for our filters for this stock.
            if not data_quality_good(symbol, financials_json, quote_json, stats_json):
                continue

            fundamentals_dict[symbol] = get_fundamental_data_for_symbol(
                symbol,
                financials_json,
                quote_json,
                stats_json
            )

        batch_idx += batch_size
    # Transform all our data into a more filterable form - a dataframe - with a bit of pandas magic.
    return pd.DataFrame.from_dict(fundamentals_dict).T

In [0]:
# We extend iexfinance a bit to support the sector collection endpoint.
class SectorCollection():

    def __init__(self, sector, **kwargs):
        self.sector = quote(sector)
        self.output_format = 'json'
        super(SectorCollection, self).__init__(**kwargs)

    @property
    def url(self):
        return '/stock/market/collection/sector?collectionName={}'.format(self.sector)

In [0]:
def update_target_securities(context):
    # In order to avoid overinvestment in large sectors, we'll be weighting each stock by its sector contribution.
    # More on this below.
    context.total_sector_contributions = 0

    # First, we'll grab all the stocks in the sectors we want to trade within.
    sector_fundamental_dfs = {}
    for sector in context.sectors:
        fundamental_df = build_sector_fundamentals(sector)
        filtered_fundamental_df = filter_fundamental_df(fundamental_df)

        # To weight by sector contribution, we'll first store each stock's
        # contribution to its own sector's presence in our portfolio as a
        # percentage.
        sector_market_cap = filtered_fundamental_df['market_cap'].sum()
        filtered_fundamental_df['sector_contribution'] = filtered_fundamental_df['market_cap'] / sector_market_cap

        # We'll add up all these percentage values, and our portfolio's weight
        # in a stock will be determined by its weight relative to the sum total
        # of all sector contributions.
        context.total_sector_contributions += filtered_fundamental_df['sector_contribution'].sum()
        sector_fundamental_dfs[sector] = filtered_fundamental_df

    # At this point, merge all our dataframes together, as we're done
    # with our sector-based calculations.
    context.all_sectors_fundamental_df = pd.concat(
        list(sector_fundamental_dfs.values()))

In [0]:
def rebalance(context):
    # We want to purchase all stocks in our dataframe, if possible.
    desired_stocks = context.all_sectors_fundamental_df.index.values

    # Exit all positions we wish to drop before starting new ones.
    for stock in context.portfolio.positions:
        if stock not in desired_stocks:
            order_target_percent(stock, 0)

    # Rebalance all stocks to target weights.
    for stock in desired_stocks:
        # Determine how much of the portfolio should be allocated to this stock.
        weight = get_weight(context, stock)
        print('weight: {}'.format(weight))
        if weight != 0:
            try:
                print('Buying {}'.format(stock))
                order_target_percent(symbol(stock), weight)
            except:
                print(
                    'Error: Tried to purchase {} but there was an error.'.format(stock))
                pass
            
            
def get_weight(context, stock):
    # As discussed above, we'll be weighting each stock by its contribution to its sector.
    return context.all_sectors_fundamental_df.loc[stock]['sector_contribution'] / context.total_sector_contributions


In [0]:
def eps_good(earnings_reports):
    # This method contains logic for filtering based on earnings reports.
    if len(earnings_reports) < 4:
        # The company must be very new. We'll skip it until it's had time to
        # prove itself.
        return False

    # earnings_reports should contain the information about the last four
    # quarterly reports.
    for report in earnings_reports:
        # We want to see consistent positive EPS.
        try:
            if not report['actualEPS']:
                return False
            if report['actualEPS'] < 0:
                return False
        except KeyError:
            # A KeyError here indicates that some data was missing or that a company is less than two years old.
            # We don't mind skipping over new companies until they've had more
            # time in the market.
            return False
    return True


def data_quality_good(symbol, financials_json, quote_json, stats_json):
    # This method makes sure that we're not going to be investing in
    # securities we don't have accurate data for.

    if len(financials_json[symbol]
           ) < 1 or quote_json[symbol]['latestPrice'] is None:
        # No recent data was found. This can sometimes happen in case of recent
        # markert suspensions.
        return False

    try:
        if not (
            quote_json[symbol]['marketCap'] and
            stats_json[symbol]['priceToBook'] and
            stats_json[symbol]['sharesOutstanding'] and
            financials_json[symbol][0]['totalAssets'] and
            financials_json[symbol][0]['currentAssets'] and
            quote_json[symbol]['latestPrice']
        ):
            # Ignore companies IEX cannot report all necessary data for, or
            # thinks are untradable.
            return False
    except KeyError:
        # A KeyError here indicates that some data we need to evaluate this
        # stock was missing.
        return False

    return True


def get_fundamental_data_for_symbol(
        symbol,
        financials_json,
        quote_json,
        stats_json):
    fundamentals_dict_for_symbol = {}

    financials = financials_json[symbol][0]

    # Calculate PB ratio.
    fundamentals_dict_for_symbol['pb_ratio'] = stats_json[symbol]['priceToBook']

    # Find the "Current Ratio" - current assets to current debt.
    current_debt = financials['currentDebt'] if financials['currentDebt'] else 1
    fundamentals_dict_for_symbol['current_ratio'] = financials['currentAssets'] / current_debt

    # Find the ratio of long term debt to short-term liquiditable assets.
    total_debt = financials['totalDebt'] if financials['totalDebt'] else 0
    fundamentals_dict_for_symbol['debt_to_liq_ratio'] = total_debt / \
        financials['currentAssets']

    # Store other information for this stock so we can filter on the data
    # later.
    fundamentals_dict_for_symbol['pe_ratio'] = quote_json[symbol]['peRatio']
    fundamentals_dict_for_symbol['market_cap'] = quote_json[symbol]['marketCap']
    fundamentals_dict_for_symbol['dividend_yield'] = stats_json[symbol]['dividendYield']

    return fundamentals_dict_for_symbol

In [0]:
# This is where we remove stocks that don't meet our investment criteria.
def filter_fundamental_df(fundamental_df):
    return fundamental_df[
        (fundamental_df.current_ratio > 1.5) &
        (fundamental_df.debt_to_liq_ratio < 1.1) &
        (fundamental_df.pe_ratio < 9) &
        (fundamental_df.pb_ratio < 1.2) &
        (fundamental_df.dividend_yield > 1.0)
    ]

# Alpaca

In [13]:
!pip3 install alpaca-trade-api

In [17]:
import alpaca_trade_api as tradeapi

api = tradeapi.REST('PK2JP8S3WBOGK3DJMLWA','kyI0YZ3e2clXNcrNnZce1QnAXBhsgUH9fwoOvrxB','https://paper-api.alpaca.markets')

# Get our account information.
account = api.get_account()

# Check if our account is restricted from trading.
if account.trading_blocked:
    print('Account is currently restricted from trading.')

# Check how much money we can use to open new positions.
print('${} is available as buying power.'.format(account.buying_power))

$396986.4 is available as buying power.


# Run

In [27]:
!pylivetrader run -f BenGraham2.ipynb

Usage: pylivetrader run [OPTIONS] [ALGOFILE]...
Try "pylivetrader run --help" for help.

Error: Invalid value for "-f" / "--file": File "BenGraham2.ipynb" does not exist.


In [21]:
!pylivetrader run --help

Usage: pylivetrader run [OPTIONS] [ALGOFILE]...

Options:
  -q, --quantopian-compatible BOOLEAN
                                  Set 0 if
                                  compatibility
                                  with the
                                  Quantopian
                                  platform is not
                                  a concern for
                                  your script.
                                  [default: True]
  --storage-engine [redis|file]   The storage
                                  engine to use to
                                  persist context.
                                  [default: file]
  -l, --log-level [INFO|CRITICAL|WARNING|DEBUG|ERROR]
                                  The minimum
                                  level of log to
                                  be written.
                                  [default: INFO]
  -r, --retry BOOLEAN             True to continue
                                  r

In [28]:
!cp /content/gdrive/My\ Drive/Colab\ Notebooks/BenGraham2.ipynb /usr/local/bin
!chmod 755 /usr/local/bin/BenGraham2.ipynb

cp: cannot stat '/content/gdrive/My Drive/Colab Notebooks/BenGraham2.ipynb': No such file or directory
chmod: cannot access '/usr/local/bin/BenGraham2.ipynb': No such file or directory


In [0]:
#Integrate Alpaca API
#Run pylivetrader
#Command code notebook: https://colab.research.google.com/drive/1QWZXsCSpX4UBiiyo0_rGy3ysEXTFmtc0